# 模型保存与恢复

Tensorflow中支持两种模型的保存。
- 检查点: 这种格式依赖于创建模型的代码。
- SavedModel：这种格式与创建模型的代码无关。

## 检查点
检查点是训练期间所创建的模型版本。
检查点目录结构如下：

--checkpoint_dir
|    |--checkpoint
|    |--model.meta
|    |--model.data-00000-of-00001
|    |--model.index

### meta文件
meta文件保存的是图的结构，meta文件是pb（protocol buffer）格式文件。

### ckpt文件
ckpt文件，保存了网络结构中所有 权重和偏置 的数值。

data与index构成ckpt文件。

.data文件保存的是变量值，

.index文件保存的是.data文件中数据和 .meta文件中结构图之间的对应关系。

在tensorflow 0.11之前，保存在**.ckpt文件中。

0.11后，通过两个文件保存,如：

model.data-00000-of-00001

model.index

### checkpoint文件
checkpoint是一个文本文件，记录了训练过程中在所有中间节点上保存的模型的名称，

首行记录的是最后（最近）一次保存的模型名称。

## 保存到检查点

检查点的最主要作用是继续训练。

使用`tf.train.Saver()`将模型保存到检查点。

需要注意的是，tensorflow变量的作用范围是在一个session里面，所以在保存模型的时候，应该在session里面通过save方法保存。

In [1]:
import tensorflow as tf

w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.constant([1, 2, 3, 4, 5], shape=[5] , name='w2')
print(w1)
print(w2)
# 使用tf.train.Saver()定义一个存储器对象
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 使用saver.save保存模型
#saver.save(sess, 'model')

# 保存相应变量到指定文件, 如果指定 global_step, 则实际保存的名称变为 model.ckpt-xxxx
# saver.save(sess, "./model.ckpt", global_step=epoch)
saver.save(sess, './ckpts/model.ckpt', global_step=1)
sess.close()

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'w1:0' shape=(2,) dtype=float32_ref>
Tensor("w2:0", shape=(5,), dtype=int32)


### 读取检查点模型
Tensorflow 模型的读取分为两种, 

一种是我们仅读取模型变量, 即 index 文件和 data 文件

另一种是读取计算图。

通常来说如果是我们自己保存的模型, 那么完全可以设置 saver.save() 函数的 write_meta_graph 参数为 False 以节省空间和保存的时间, 

因为我们可以使用已有的代码直接重新构建计算图. 当然如果为了模型迁移到其他地方, 则最好同时保存变量和计算图.

#### 读取计算图
读取模型权重也很简单, 仍然使用 tf.train.Saver() 来读取:

从 meta 文件读取计算图使用 tf.train.import_meta_graph() 函数, 比如:

In [2]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('./ckpts/model.ckpt-1.meta')

此时计算图就会加载到 sess 的默认计算图中, 这样我们就无需再次使用大量的脚本来定义计算图了。

实际上使用上面这两行代码即可完成计算图的读取。

**注意**可能我们获取的模型(meta文件)同时包含定义在CPU主机(host)和GPU等设备(device)上的, 上面的代码保留了原始的设备信息。

此时如果我们想同时加载模型权重, 那么如果当前没有指定设备的话就会出现错误, 因为tensorflow无法按照模型中的定义把某些变量(的值)放在指定的设备上。

那么有一个办法是增加一个参数清除设备信息。

In [3]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('./ckpts/model.ckpt-1.meta', clear_devices=True)

#### 读取模型权重
读取模型权重也很简单, 仍然使用 tf.train.Saver() 来读取:

In [4]:
with tf.Session() as sess:
    saver.restore(sess, './ckpts/model.ckpt-1')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./ckpts/model.ckpt-1


注意模型路径中应当以诸如 .ckpt 之类的来结尾, 即需要保证实际存在的文件是 model.ckpt.data-00000-of-00001 和 model.ckpt.index , 而指定的路径是 model.ckpt 即可。

注意, 载入的模型变量是不需要再初始化的(即不需要 tf.variable_initializer() 初始化)

### 冻结模型
我们冻结模型的目的是不再训练, 而仅仅做正向推导使用, 

所以才会把变量转换为常量后同计算图结构保存在协议缓冲区文件(.pb)中, 因此需要在计算图中预先定义输出节点的名称.

In [5]:
# 指定模型输出, 这样可以允许自动裁剪无关节点. 这里认为使用逗号分割
output_nodes = ['w1', 'w2']

# 1. 加载模型
saver = tf.train.import_meta_graph('./ckpts/model.ckpt-1.meta', clear_devices=True)

with tf.Session(graph=tf.get_default_graph()) as sess:
    # 序列化模型
    input_graph_def = sess.graph.as_graph_def()
    # 2. 载入权重
    saver.restore(sess, './ckpts/model.ckpt-1')
    # 3. 转换变量为常量
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess,
                                                                    input_graph_def,
                                                                    output_nodes)
    # 4. 写入文件
    with open('frozen_model.pb', 'wb') as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from ./ckpts/model.ckpt-1
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 1 variables.
INFO:tensorflow:Converted 1 variables to const ops.


#### 调用模型

模型的执行过程也很简单, 首先从协议缓冲区文件(*.pb)中读取模型, 然后导入计算图

In [6]:
frozen_graph_path = './frozen_model.pb'
# 读取模型并保存到序列化模型对象中
with open(frozen_graph_path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

# 导入计算图
graph = tf.Graph()
with graph.as_default():
    tf.import_graph_def(graph_def, name="MyGraph")
    print([tensor.name for tensor in tf.get_default_graph().as_graph_def().node])


['MyGraph/w1', 'MyGraph/w2']


之后就是获取输入和输出的张量对象, 

注意, 在 Tensorflow的计算图结构中, 我们只能使用 feed_dict 把数值数组传入张量 Tensor , 同时也只能获取张量的值, 而不能给Operation 赋值. 由于我们导入序列化模型到计算图时给定了 name 参数, 所以导入所有操作都会加上 MyGraph 前缀.

接下来我们获取输入和输出对应的张量:

In [7]:
with graph.as_default():
    x_tensor = graph.get_tensor_by_name('MyGraph/w1:0')
    with tf.Session() as sess:
        print(sess.run(x_tensor))

[-0.51579756 -0.27759436]
